<a href="https://colab.research.google.com/github/kiitaamuuraa/Asobiba/blob/main/Japanse_BERT_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

     |████████████████████████████████| 1.9MB 6.5MB/s 
     |████████████████████████████████| 3.2MB 22.8MB/s 
     |████████████████████████████████| 890kB 41.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a671236514b2d59f9e747bb6795c84401d50196921ab329245dc8397aba6d131
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!pip install transformers["ja"]

     |████████████████████████████████| 47.4MB 61kB/s 
     |████████████████████████████████| 13.4MB 242kB/s 
     |████████████████████████████████| 491kB 36.1MB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-cp37-none-any.whl size=47658825 sha256=3d4d5f9a2be11b47c743e0972faa585f7f71c382de6909543afe3cce8d081139
  Stored in directory: /root/.cache/pip/wheels/20/48/8d/b66d8361a27f58f41ec86640e4fd2640de0403a6367511eab7
  Created wheel for unidic: filename=unidic-1.0.3-cp37-none-any.whl size=5497 sha256=b7eae0b41dbab3c0ab5b11e26a6ea34765feff6f9b22aafe951bad961666114a
  Stored in directory: /root/.cache/pip/wheels/d3/26/e2/fb76c79fd14391eb994eab021c9129c24814125298e1e5b96a
  Created wheel for ipadic: filename=ipadic-1.0.0-cp37-none-any.whl size=13556725 sha256=c6dfd8b847373504900d28cdc66167349f8d7299c3be6a476791abcb6d6157de
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
Successfully built unidic-lite unidic ipadic

In [3]:
# PyTorch
import torch
import torch.nn as nn

# transformers
from transformers import BertModel
from transformers import BertJapaneseTokenizer

## 1. インスタンス作成

In [80]:
# トークナイザー
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')
# モデルインスタンス
bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese')

## 2. 前処理

In [40]:
kanikosen= ['「おい地獄さ行ぐんだで！」',\
            '二人はデッキの手すりに寄りかかって、蝸牛が背のびをしたように延びて、海を抱かえ込んでいる函館の街を見ていた。',\
            '――漁夫は指元まで吸いつくした煙草を唾と一緒に捨てた。']

# トークナイズ
# 純粋にトークナイズした結果で、BERTの入力形式ではないので注意
# tokeneizerメソッドはバッチを受け付けないので注意

for sent in kanikosen:
    print(tokenizer.tokenize(sent))
    print(len(sent), '\n')

['「', 'おい', '地獄', 'さ', '行', 'ぐん', '##だ', '##で', '!」']
13 

['二', '人', 'は', 'デッキ', 'の', '手', '##すり', 'に', '寄り', '##かかっ', 'て', '、', '[UNK]', 'が', '背', '##の', '##び', 'を', 'し', 'た', 'よう', 'に', '延び', 'て', '、', '海', 'を', '抱か', 'え', '込ん', 'で', 'いる', '函館', 'の', '街', 'を', '見', 'て', 'い', 'た', '。']
54 

['――', '漁', '##夫', 'は', '指', '元', 'まで', '吸い', 'つく', '##し', 'た', '煙草', 'を', '[UNK]', 'と', '一緒', 'に', '捨て', 'た', '。']
27 



In [45]:
# BERT入力形式に変換
tokenized = tokenizer(\
    kanikosen,             # 入力文
    padding=True,          # 入力文集合中の最大長に合わせてパディング
    truncation=True,       # 長すぎる文を、以下のmax_lengthのサイズに合わせてカットするか
    max_length=512,
    return_tensors="pt"    # 返却する型: 省略時はリスト、他はtf,np
    )

In [46]:
# 確認
for k, v in tokenized.items():
    print('key: ', k)
    print('value: ', v, '\n')

for sent in tokenized['input_ids']:
    print(len(sent))
decoded = str()

for id in tokenized['input_ids'][0]:
    decoded += tokenizer.decode(id)
print(decoded)

key:  input_ids
value:  tensor([[    2,    36,  7613,  9867,    26,    77, 11043, 28565, 28455,  3286,
             3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [    2,   287,    53,     9, 12824,     5,   319, 26853,     7,  6562,
         24645,    16,     6,     1,    14,  1503, 28444, 28670,    11,    15,
            10,   124,     7, 13081,    16,     6,   295,    11, 27978,  1723,
         19736,    12,    33,  8159,     5,  1243,    11,   212,    16,    21,
            10,     8,     3],
        [    2, 18454,  3114, 29219,     9,   254,   281,   126, 16470,  2950,
         28454,    10, 24168,    11,     1,    13,  4265,     7,  6638,    10,
             8,     3,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

## 3. 学習

### 構成概要
Embedding層 → BERT層 → pool層  
以下は、 ```print(bert)``` の出力結果の概要

### Embedding層

```python
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
```
### BERT層
以下の層が12積層
```python
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
        )
        (output): BertOutput(
          (dense): Linear(in_features=3072, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
```
### pool層

```python

  (pooler): BertPooler(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
  )
)
```

In [82]:
# pool層に分類層を追加
bert.pooler = nn.Sequential(bert.pooler, nn.Linear(768, 3))

In [83]:
print(bert.pooler)

Sequential(
  (0): BertPooler(
    (dense): Linear(in_features=768, out_features=768, bias=True)
    (activation): Tanh()
  )
  (1): Linear(in_features=768, out_features=3, bias=True)
)


In [84]:
# モデルを学習モードに切り替え
bert.train()

outputs = bert(tokenized['input_ids'], attention_mask=tokenized['attention_mask'])

In [86]:
for k, v in outputs.items():
    print('key: ', k, 'shape: ', v.shape)
print(outputs[1])

key:  last_hidden_state shape:  torch.Size([3, 43, 768])
key:  pooler_output shape:  torch.Size([3, 3])
tensor([[ 0.2459, -0.1721, -0.0784],
        [ 0.1571, -0.1872,  0.0925],
        [ 0.2502, -0.2422, -0.2857]], grad_fn=<AddmmBackward>)


In [89]:
# オプティマイザ
from transformers import AdamW
optimizer = AdamW(bert.parameters(), lr=1e-5)

In [90]:
# ここから先は、いつものPyTorch（もろもろ省略はしている）
labels = torch.tensor([0, 1, 2])
criterion = nn.CrossEntropyLoss()
loss = criterion(outputs[1], labels)

tensor(1.1693, grad_fn=<NllLossBackward>)